In [12]:
!pip install pandas numpy yfinance matplotlib seaborn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

file_path = '//content/drive/MyDrive/tech-stock-2019-2024.gsheet'

df = pd.read_csv(file_path)

df.info()

KeyboardInterrupt: 